In [1]:
import random
import re
import numpy as np
import pandas as pd

In [74]:
np.set_printoptions(linewidth=150)

In [3]:
base_damage = 5
build1 = '109060606060606060606060601050606010010010010010010010010010010010010010010010010010050551051010050051010043040010010010010043300040552010550552010572430010010010050301041550552010550552010562420010010010240242010550552010550552010562410010010010240242010550552010550552010562360010010010240242010550552010550552010562210010010010240242010570552010550552010562260010010010240242010550552010550552010552160010010010053052010280552010550552010552160010010010010010010280042551041572010552510010010010010030010053153153651572010552480010010010010053363163163453343052010642470010010010010010010010010010010010010010020010010'
build2 = '105050506050705060501050605050506010010010010010010010010010010010010010010010043243243413363213243243243153040010010010242043563563563563563563563040620010010010242562043553553553553553040560240010010010242562552043543553553040550560440010010010242562552042541451051550550560620010010010242562552010010010162550550560160010010010572562552010010010162550550560160010010010572562552010030010362550550560160010010010572562552010053433052550550560160010010010572562042551551551551041550560510010010010562042561561551551551551041560470010010010042561561571571561561561561041480010010010010010010010010010010010010010020010010'
build = build2

In [4]:
def parse_build_old(build):
    custom = build[0:1] # custom ship Y/N
    custom_blocks = build[1:33] # custom blocks
    custom_block_list = re.findall('..?',custom_blocks)
    custom_block_name_2d_array = np.reshape(custom_block_mapper(custom_block_list), (4,4))
    custom_block_id_2d_array = np.reshape(custom_block_list, (4,4))
    items = build[33:] #items
    items_and_orientation = re.findall('...?',items)
    item_list = pd.Series(items_and_orientation).str[0:2].values
    item_list_named_2d_array = ...
    item_list_2d_array = np.reshape(item_list, (14,14))
    return custom_block_list, custom_block_name_2d_array, custom_block_id_2d_array, item_list, item_list_2d_array

In [19]:
def parse_build(build):
    custom = build[0:1] # custom ship Y/N
    custom_blocks = build[1:33] # custom blocks
    custom_block_list = re.findall('..?',custom_blocks)
    custom_block_name_2d_array = np.reshape(custom_block_mapper(custom_block_list), (4,4))
    custom_block_id_2d_array = np.reshape(custom_block_list, (4,4))
    items = build[33:] #items
    items_and_orientation = re.findall('...?',items)
    items_and_orientation_2d_array = np.reshape(items_and_orientation, (14,14))
#     item_list = pd.Series(items_and_orientation).str[0:2].values
#     item_list_named_2d_array = ...
#     item_list_2d_array = np.reshape(item_list, (14,14))
    return custom_block_list, custom_block_name_2d_array, custom_block_id_2d_array, items_and_orientation_2d_array

In [5]:
def chance(percentage):
    chance = random.randint(0,percentage)
    return chance

In [6]:
def custom_block_mapper(custom_block_number):
    custom_block_mapping = {'01':'ejection_up', '02':'auto_fly', '03':'second_try'
                        , '04':'manual_burst', '05':'3x3', '06':'blue'
                        , '07':'longer_countdown', '08':'light_green', '09':'skipp'
                        , '10':'10%_more', '11':'ejection_side', '12':'red_after_wave'}
    custom_block_element_name = [custom_block_mapping[x] for x in custom_block_number]
    return custom_block_element_name

In [7]:
class item:
    def __init__(self, id, orientation):
        self.id = id
        self.orientation = orientation

In [8]:
def slicer_vectorized(a,start,end):
    b = a.view((str,1)).reshape(len(a),-1)[:,start:end]
    return np.fromstring(b.tostring(),dtype=(str,end-start))

A zip funkció jó lehet nekem az item lista darabolására array-be.
https://stackoverflow.com/questions/48633549/splitting-a-list-of-strings-with-a-delimiter-and-adding-to-new-lists

DEV ZONE

In [75]:
items_and_orientation_2d_array

array([['010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010'],
       ['010', '043', '243', '243', '413', '363', '213', '243', '243', '243', '153', '040', '010', '010'],
       ['010', '242', '043', '563', '563', '563', '563', '563', '563', '563', '040', '620', '010', '010'],
       ['010', '242', '562', '043', '553', '553', '553', '553', '553', '040', '560', '240', '010', '010'],
       ['010', '242', '562', '552', '043', '543', '553', '553', '040', '550', '560', '440', '010', '010'],
       ['010', '242', '562', '552', '042', '541', '451', '051', '550', '550', '560', '620', '010', '010'],
       ['010', '242', '562', '552', '010', '010', '010', '162', '550', '550', '560', '160', '010', '010'],
       ['010', '572', '562', '552', '010', '010', '010', '162', '550', '550', '560', '160', '010', '010'],
       ['010', '572', '562', '552', '010', '030', '010', '362', '550', '550', '560', '160', '010', '010'],
       ['010', '572', '562', '552', '

In [105]:
def get_starting_position(items_and_orientation_2d_array):
    starting_position = np.where(items_and_orientation_2d_array=='020')
    x0 = np.asarray(starting_position).T[0][0]
    y0 = np.asarray(starting_position).T[0][1]
    return x0, y0

In [96]:
directions = (0,1,2,3)
def turn(id, turn):
    # 0 - up, 1 - right, 2 - down, 3 - left
    match turn:
        case 'turn_left':
            id -= 1
            if id < 0:
                id = 3
        case 'turn_right':
            id += 1
            if id > 3:
                id = 0
    return directions[id]

In [115]:
def next_coordination(x_coord, y_coord, item_with_orientation):
    item = item_with_orientation[0:2]
    orientation = item_with_orientation[2]
    match item:
        case '04':
            direction = turn(orientation,'turn_left')
        case '05':
            direction = turn(orientation,'turn_right')
        case other:
            direction = orientation
    match orientation:
        case '0':
            y_coord -= 1 #rossz id-t módosítok, az x a 13, és azt kell csökkenteni 0-nál. Átírni, vagy a variable is lehetne más, mondjuk h_coord, v_coord.
        case '1':
            x_coord += 1
        case '2':
            y_coord += 1
        case '3':
            x_coord -= 1
    return x_coord, y_coord

In [107]:
starting_position = get_starting_position(items_and_orientation_2d_array)

In [108]:
starting_position

(13, 11)

In [110]:
current_x = starting_position[0]
current_y = starting_position[1]

In [114]:
items_and_orientation_2d_array[current_x,current_y][2]

'0'

In [129]:
next_x, next_y = next_coordination(current_x, current_y, items_and_orientation_2d_array[current_x,current_y])
print(next_x, next_y)
print(items_and_orientation_2d_array[current_x,current_y])

13 10
020


In [137]:
next_x, next_y = next_coordination(next_x, next_y, items_and_orientation_2d_array[next_x,next_y])
print(next_x, next_y)
print(items_and_orientation_2d_array[next_x,next_y])

13 2
010


In [138]:
np.where(items_and_orientation_2d_array=='153')

(array([1], dtype=int64), array([10], dtype=int64))

In [79]:
i = 'a'
match i:
    case '04':
        print('turn_left')
    case '05':
        print('turn_right')
    case other:
        print('other')

other


In [ ]:
def convert_array_to_list(items_and_orientation_2d_array):
    x0, y0 = get_starting_position(items_and_orientation_2d_array)
    next_coordination(x0, y0, item)
    item_list = ()
    add_item

In [ ]:
def add_item(coordination, array):
    

In [25]:
starting_position = np.where(items_and_orientation_2d_array=='020')
print(starting_position)

(array([13], dtype=int64), array([11], dtype=int64))


In [26]:
items_and_orientation_2d_array[13][11]

'020'

In [46]:
x = np.asarray(starting_position).T[0][0]
y= np.asarray(starting_position).T[0][1]

In [47]:
print(x ,y)

13 11


In [32]:
type(starting_position)

tuple

In [35]:
starting_position.item(1)

AttributeError: 'tuple' object has no attribute 'item'

In [34]:
type(starting_position[0])

numpy.ndarray

In [31]:
items_and_orientation_2d_array[starting_position]

array(['020'], dtype='<U3')

In [63]:
np.append(np.array([0,1]),[1])

array([0, 1, 1])

In [65]:
ex1 = np.array([0,1])
ex2 = np.array([2,3])
ex3 = np.array([4,5])

In [70]:
# a split item működhetne valahogy így:
np.stack((np.append(ex1,ex2), np.append(ex1,ex3)))

array([[0, 1, 2, 3],
       [0, 1, 4, 5]])

In [68]:
np.stack(ex1,ex2)

TypeError: only integer scalar arrays can be converted to a scalar index

In [18]:
items_and_orientation_2d_array

array([['010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010', '010',
        '010'],
       ['010', '043', '243', '243', '413', '363', '213', '243', '243', '243', '153', '040', '010',
        '010'],
       ['010', '242', '043', '563', '563', '563', '563', '563', '563', '563', '040', '620', '010',
        '010'],
       ['010', '242', '562', '043', '553', '553', '553', '553', '553', '040', '560', '240', '010',
        '010'],
       ['010', '242', '562', '552', '043', '543', '553', '553', '040', '550', '560', '440', '010',
        '010'],
       ['010', '242', '562', '552', '042', '541', '451', '051', '550', '550', '560', '620', '010',
        '010'],
       ['010', '242', '562', '552', '010', '010', '010', '162', '550', '550', '560', '160', '010',
        '010'],
       ['010', '572', '562', '552', '010', '010', '010', '162', '550', '550', '560', '160', '010',
        '010'],
       ['010', '572', '562', '552', '010', '030', '010', '362', '550', '550', '5

In [20]:
custom_block_list, custom_block_name_2d_array, custom_block_id_2d_array, items_and_orientation_2d_array = parse_build(build)
print(custom_block_name_2d_array)
print(custom_block_id_2d_array)
print(items_and_orientation_2d_array)

[['3x3' '3x3' '3x3' 'blue']
 ['3x3' 'longer_countdown' '3x3' 'blue']
 ['3x3' 'ejection_up' '3x3' 'blue']
 ['3x3' '3x3' '3x3' 'blue']]
[['05' '05' '05' '06']
 ['05' '07' '05' '06']
 ['05' '01' '05' '06']
 ['05' '05' '05' '06']]
[['010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010']
 ['010' '043' '243' '243' '413' '363' '213' '243' '243' '243' '153' '040' '010' '010']
 ['010' '242' '043' '563' '563' '563' '563' '563' '563' '563' '040' '620' '010' '010']
 ['010' '242' '562' '043' '553' '553' '553' '553' '553' '040' '560' '240' '010' '010']
 ['010' '242' '562' '552' '043' '543' '553' '553' '040' '550' '560' '440' '010' '010']
 ['010' '242' '562' '552' '042' '541' '451' '051' '550' '550' '560' '620' '010' '010']
 ['010' '242' '562' '552' '010' '010' '010' '162' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '552' '010' '010' '010' '162' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '552' '010' '030' '010' '362' '550' '550' '560' '160' '01

In [22]:
pd.Series(item_list).str[0:2].values

array(['01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01',
       '04', '24', '24', '41', '36', '21', '24', '24', '24', '15', '04', '01', '01', '01', '24',
       '04', '56', '56', '56', '56', '56', '56', '56', '04', '62', '01', '01', '01', '24', '56',
       '04', '55', '55', '55', '55', '55', '04', '56', '24', '01', '01', '01', '24', '56', '55',
       '04', '54', '55', '55', '04', '55', '56', '44', '01', '01', '01', '24', '56', '55', '04',
       '54', '45', '05', '55', '55', '56', '62', '01', '01', '01', '24', '56', '55', '01', '01',
       '01', '16', '55', '55', '56', '16', '01', '01', '01', '57', '56', '55', '01', '01', '01',
       '16', '55', '55', '56', '16', '01', '01', '01', '57', '56', '55', '01', '03', '01', '36',
       '55', '55', '56', '16', '01', '01', '01', '57', '56', '55', '01', '05', '43', '05', '55',
       '55', '56', '16', '01', '01', '01', '57', '56', '04', '55', '55', '55', '55', '04', '55',
       '56', '51', '01', '01',

In [ ]:
def item_mapper(item_list):
    item_mapping = {'010'
        

}

In [27]:
print(np.reshape(item_list, (14,14)))

[['010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010']
 ['010' '043' '243' '243' '413' '363' '213' '243' '243' '243' '153' '040' '010' '010']
 ['010' '242' '043' '563' '563' '563' '563' '563' '563' '563' '040' '620' '010' '010']
 ['010' '242' '562' '043' '553' '553' '553' '553' '553' '040' '560' '240' '010' '010']
 ['010' '242' '562' '552' '043' '543' '553' '553' '040' '550' '560' '440' '010' '010']
 ['010' '242' '562' '552' '042' '541' '451' '051' '550' '550' '560' '620' '010' '010']
 ['010' '242' '562' '552' '010' '010' '010' '162' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '552' '010' '010' '010' '162' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '552' '010' '030' '010' '362' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '552' '010' '053' '433' '052' '550' '550' '560' '160' '010' '010']
 ['010' '572' '562' '042' '551' '551' '551' '551' '041' '550' '560' '510' '010' '010']
 ['010' '562' '042' '561' '561' '551' '551'

In [6]:
custom = build[0:1] # custom ship Y/N
custom_blocks = build[1:33] # custom blocks
custom_block_list = re.findall('..?',custom_blocks)
custom_block_name_2d_array = np.reshape(custom_block_mapper(custom_block_list), (4,4))
custom_block_id_2d_array = np.reshape(custom_block_list, (4,4))
items = build[33:] #items
item_list = re.findall('...?',items)

In [ ]:
item_mapping = {

}

In [17]:
item_list

['010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '050',
 '551',
 '051',
 '010',
 '050',
 '051',
 '010',
 '043',
 '040',
 '010',
 '010',
 '010',
 '010',
 '043',
 '300',
 '040',
 '552',
 '010',
 '550',
 '552',
 '010',
 '572',
 '430',
 '010',
 '010',
 '010',
 '050',
 '301',
 '041',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '420',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '410',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '360',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '210',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '570',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '260',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '552',
 '160',
 '010',


to do:
item_list-re csinálni mappelőt.
megírni a kezdő pontot az array-re.
megírni egy functiont ami elindul a path-on és úgy teszi sorba az itemeket egy listába
a direction-t kitalálni, hogy melyik mi. Gondolom a 0 az up.


In [18]:
print(np.reshape(item_list, (14,14)))

[['010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010' '010']
 ['010' '010' '010' '050' '551' '051' '010' '050' '051' '010' '043' '040' '010' '010']
 ['010' '010' '043' '300' '040' '552' '010' '550' '552' '010' '572' '430' '010' '010']
 ['010' '050' '301' '041' '550' '552' '010' '550' '552' '010' '562' '420' '010' '010']
 ['010' '240' '242' '010' '550' '552' '010' '550' '552' '010' '562' '410' '010' '010']
 ['010' '240' '242' '010' '550' '552' '010' '550' '552' '010' '562' '360' '010' '010']
 ['010' '240' '242' '010' '550' '552' '010' '550' '552' '010' '562' '210' '010' '010']
 ['010' '240' '242' '010' '570' '552' '010' '550' '552' '010' '562' '260' '010' '010']
 ['010' '240' '242' '010' '550' '552' '010' '550' '552' '010' '552' '160' '010' '010']
 ['010' '053' '052' '010' '280' '552' '010' '550' '552' '010' '552' '160' '010' '010']
 ['010' '010' '010' '010' '280' '042' '551' '041' '572' '010' '552' '510' '010' '010']
 ['010' '010' '030' '010' '053' '153' '153'

In [9]:
custom_block_mapping = {'01':'ejection_up', '02':'auto_fly', '03':'second_try'
                        , '04':'manual_burst', '05':'3x3', '06':'blue'
                        , '07':'longer_countdown', '08':'light_green', '09':'skipp'
                        , '10':'10%_more', '11':'ejection_side', '12':'red_after_wave'}

In [13]:
custom_block_2d_array = np.reshape(custom_block_list, (4,4))
custom_block_2d_array

array([['09', '06', '06', '06'],
       ['06', '06', '06', '06'],
       ['06', '06', '06', '06'],
       ['01', '05', '06', '06']], dtype='<U2')

In [20]:
#3d array transformation for custom ship blocks
np.reshape(custom_block_mapper(custom_block_list), (4,4))

array([['skipp', 'blue', 'blue', 'blue'],
       ['blue', 'blue', 'blue', 'blue'],
       ['blue', 'blue', 'blue', 'blue'],
       ['ejection_up', '3x3', 'blue', 'blue']], dtype='<U11')

In [ ]:
def custom_block_mapper(custom_block_number):
    custom_block_mapping = {'01':'ejection_up', '02':'auto_fly', '03':'second_try'
                        , '04':'manual_burst', '05':'3x3', '06':'blue'
                        , '07':'longer_countdown', '08':'light_green', '09':'skipp'
                        , '10':'10%_more', '11':'ejection_side', '12':'red_after_wave'}
    custom_block_element_name = [custom_block_mapping[x] for x in custom_block_number]
    return custom_block_element_name

In [ ]:
item_list

['010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '010',
 '050',
 '551',
 '051',
 '010',
 '050',
 '051',
 '010',
 '043',
 '040',
 '010',
 '010',
 '010',
 '010',
 '043',
 '300',
 '040',
 '552',
 '010',
 '550',
 '552',
 '010',
 '572',
 '430',
 '010',
 '010',
 '010',
 '050',
 '301',
 '041',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '420',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '410',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '360',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '210',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '570',
 '552',
 '010',
 '550',
 '552',
 '010',
 '562',
 '260',
 '010',
 '010',
 '010',
 '240',
 '242',
 '010',
 '550',
 '552',
 '010',
 '550',
 '552',
 '010',
 '552',
 '160',
 '010',


In [ ]:
item_list = re.findall('...?',items)

In [ ]:
build_example[33:] items

'010010010010010010010010010010010010010010010010010050551051010050051010043040010010010010043300040552010550552010572430010010010050301041550552010550552010562420010010010240242010550552010550552010562410010010010240242010550552010550552010562360010010010240242010550552010550552010562210010010010240242010570552010550552010562260010010010240242010550552010550552010552160010010010053052010280552010550552010552160010010010010010010280042551041572010552510010010010010030010053153153651572010552480010010010010053363163163453343052010642470010010010010010010010010010010010010010020010010'